# Graph  elements

## State

In [1]:
from typing import TypedDict



class State(TypedDict):
    graph_state:str

## Nodes
just a python function with the first argument is state.

Each node will override the prior state value.

In [2]:
def node_1(state):
    print("__NODE1__")
    return dict(graph_state=state['graph_state'] +  "IIII am")
def node_2(state):
    print("__NODE2__")
    return dict(graph_state=state['graph_state'] +  "sad")
def node_3(state):
    print("__NODE3__")
    return dict(graph_state=state['graph_state'] +  "happy")


## Edges
Simply connect nodes

Normal edges and conditional edges.

In [3]:
import random
from typing import Literal
def decide_node(state) -> Literal["node_2","node_3"]:
    state = state['graph_state']
    if random.random()<0.5:
        return "node_2"
    return "node_3"

## Graph Construction
Steps:
1. Initialize StateGraph with State class we define above.
2. Add Nodes and Edges, and special START and END nodes

In [4]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image,display
builder = StateGraph(State)

builder.add_node("node_1",node_1)
builder.add_node("node_2",node_2)
builder.add_node("node_3",node_3)

builder.add_edge(START,"node_1")
builder.add_conditional_edges("node_1",decide_node)
builder.add_edge("node_2",END)
builder.add_edge("node_3",END)

graph = builder.compile()
draw_graph = graph.get_graph()

In [5]:
# display(Image(draw_graph.draw_mermaid_png())) # wait for draw mermaid

In [6]:
graph.invoke(dict(graph_state = "Hi im Hieu"))

__NODE1__
__NODE2__


{'graph_state': 'Hi im HieuIIII amsad'}

# Message State
We want when messages go out a node, new message is APPEND to old messages state. And node is UPDATE (OVERRIDE) prior state values. So we use Reducer.




In [7]:
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages, MessagesState
from transformers import TextGenerationPipeline
# Reducer
history = [AIMessage("Hello i am AI."),
            HumanMessage("Tell me a joke.")]
new_message = AIMessage(" OK then ...")
history = add_messages(history,new_message)
print(history)

# Inside prebuilt MessagesState
# from typing_extensions import TypedDict
# class MessageState(TypedDict):
#     messages: Annotated[ list[AnyMessage], add_messages ]

# We will use the prebuilt one like this.
class State(MessagesState):
    pass



[AIMessage(content='Hello i am AI.', additional_kwargs={}, response_metadata={}, id='35ee5458-2205-4002-98fd-42d5cb24fcfe'), HumanMessage(content='Tell me a joke.', additional_kwargs={}, response_metadata={}, id='68431fa1-1ace-4020-9937-4e9a8de599fa'), AIMessage(content=' OK then ...', additional_kwargs={}, response_metadata={}, id='5c741a69-54bd-4e6f-a483-56c1151f6f40')]
